In [ ]:
!pip install catboost

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from catboost import CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=100, 
    #task_type = 'GPU'
    ,logging_level = 'Silent'
)

from sklearn.model_selection import KFold
kf = KFold(n_splits=3, shuffle=True, random_state=0)

In [ ]:
from sklearn.metrics import accuracy_score
from itertools import combinations

In [ ]:
def cv_cat(train):
  result = []
  X = train[train.columns.difference(['y'])]
  y = train.loc[:, 'y']
  for k, (train_index,test_index) in enumerate(kf.split(data)):
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index, :], X.iloc[test_index,:]
    cat_fit = cb_model.fit(X_train, y_train, cat_features=list(train.select_dtypes(include=['object']).columns))
    pred = cat_fit.predict(X_valid)
    true = y_valid.to_numpy().flatten()
    error = accuracy_score(true, pred)
    result.append(error)
  return (np.mean(result))

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
X,y = make_classification(n_samples=1000, n_features=20, n_informative=8,n_redundant=2 ,n_repeated=0, n_classes=2, class_sep=1.0, hypercube=True,scale=1.0, shuffle=False, random_state=None)

In [ ]:
data = pd.DataFrame(X)
data.columns = ['x'+str(i) for i in range(data.shape[1])]
data['y'] = y

In [ ]:
feature_list = list(set(data.columns.tolist())-set('y'))
used_feature = []
unused_feature = feature_list
num_feature = 1
direction = "up"
step_size = 0
best_score = 0
while num_feature <= data.shape[1]:
  if (direction=="up"):
    var_comb1 = [t+tuple(used_feature) for t in list(combinations(unused_feature, 1))]
  else:
    var_comb1 = list(combinations(used_feature,num_feature-1))
  if (num_feature>1):
    var_comb2 = list((used_feature,))
  else: 
    var_comb2 = [] 
  variable_df = pd.DataFrame()
  variable_df['variable'] = var_comb1+var_comb2
  variable_df['ACC'] = 0

  for k in range(variable_df.shape[0]):
    using_var = list(variable_df.iloc[k,0]+('y',))
    variable_df.iloc[k,1] = cv_cat(data.loc[:, using_var])
    print(using_var,k,variable_df.iloc[k,1])

  idx = [int(variable_df['ACC'].idxmax()),]
  min_variable = variable_df.iloc[idx].iloc[0,0]

  if (direction=='down' and best_score==np.round(variable_df['ACC'].max(),6) and step_size==0):
    break
  best_score = np.round(variable_df['ACC'].max(),6)

  if (len(min_variable)==num_feature):
    num_feature = num_feature+1
    new_direction = "up"
  else:
    num_feature = num_feature-1
    new_direction = "down"
    
  if (direction==new_direction):
    step_size += 1
  if (direction!=new_direction):
    step_size = 0
  print(direction,new_direction,best_score,step_size)
  direction = new_direction
  used_feature = min_variable
  unused_feature = list(set(feature_list)-set(used_feature))
  print(num_feature, used_feature)

['x18', 'y'] 0 0.4990379601158044
['x13', 'y'] 1 0.495000989012965
['x16', 'y'] 2 0.5239970509431587
['x9', 'y'] 3 0.521021021021021
['x15', 'y'] 4 0.49099698500896105
['x7', 'y'] 5 0.7189944435453418
['x5', 'y'] 6 0.5850371329413245
['x11', 'y'] 7 0.4870139600678523
['x10', 'y'] 8 0.5050289810768852
['x19', 'y'] 9 0.4859680039320758
['x12', 'y'] 10 0.5150240060419701
['x4', 'y'] 11 0.5200469931008853
['x3', 'y'] 12 0.4830309351267435
['x2', 'y'] 13 0.6480042917168666
['x8', 'y'] 14 0.7190064315812817
['x0', 'y'] 15 0.5000179820539102
['x1', 'y'] 16 0.5500200799601997
['x6', 'y'] 17 0.5819981658305012
['x17', 'y'] 18 0.4740189291087494
['x14', 'y'] 19 0.503006000011988
up up 0.719006 1
2 ('x8',)
['x18', 'x8', 'y'] 0 0.7110104116092139
['x16', 'x8', 'y'] 1 0.7279794764824704
['x9', 'x8', 'y'] 2 0.7749935564306822
['x13', 'x8', 'y'] 3 0.7219854585124046
['x15', 'x8', 'y'] 4 0.7199834565104025
['x7', 'x8', 'y'] 5 0.7789765813717908
['x5', 'x8', 'y'] 6 0.7960205714696733
['x11', 'x8', 'y']

In [ ]:
variable_df

,variable,ACC
0,"(x18, x0, x2, x1, x3, x7, x5)",0.939017
1,"(x18, x0, x2, x1, x3, x7, x8)",0.912005
2,"(x18, x0, x2, x1, x3, x5, x8)",0.926004
3,"(x18, x0, x2, x1, x7, x5, x8)",0.922018
4,"(x18, x0, x2, x3, x7, x5, x8)",0.920010
5,"(x18, x0, x1, x3, x7, x5, x8)",0.918017
6,"(x18, x2, x1, x3, x7, x5, x8)",0.918017
7,"(x0, x2, x1, x3, x7, x5, x8)",0.941010
8,"(x18, x0, x2, x1, x3, x7, x5, x8)",0.946021
